In [ ]:
from json import dumps, loads
from typing import List
from collections import defaultdict


# Dinic's algorithm
from collections import deque
class Dinic:
    def __init__(self, N):
        self.N = N
        self.G = [[] for i in range(N)]

    def add_edge(self, fr, to, cap):
        forward = [to, cap, None]
        forward[2] = backward = [fr, 0, forward]
        self.G[fr].append(forward)
        self.G[to].append(backward)

    def add_multi_edge(self, v1, v2, cap1, cap2):
        edge1 = [v2, cap1, None]
        edge1[2] = edge2 = [v1, cap2, edge1]
        self.G[v1].append(edge1)
        self.G[v2].append(edge2)

    def bfs(self, s, t):
        self.level = level = [None]*self.N
        deq = deque([s])
        level[s] = 0
        G = self.G
        while deq:
            v = deq.popleft()
            lv = level[v] + 1
            for w, cap, _ in G[v]:
                if cap and level[w] is None:
                    level[w] = lv
                    deq.append(w)
        return level[t] is not None

    def dfs(self, v, t, f):
        if v == t:
            return f
        level = self.level
        for e in self.it[v]:
            w, cap, rev = e
            if cap and level[v] < level[w]:
                d = self.dfs(w, t, min(f, cap))
                if d:
                    e[1] -= d
                    rev[1] += d
                    return d
        return 0

    def flow(self, s, t):
        flow = 0
        INF = 10**9 + 7
        G = self.G
        while self.bfs(s, t):
            *self.it, = map(iter, self.G)
            f = INF
            while f:
                f = self.dfs(s, t, INF)
                flow += f
        return flow
    
def max_score(n_mutants: int, wishlist_y: dict, wishlist_z: dict, wishlist_mutants: List[dict], friendships: List[dict]) -> int:
    D = Dinic(n_mutants + 2)
    tmp_dict = defaultdict(int)
    ans = 0
    for mutant_num, priority in wishlist_y.items():
        tmp_dict[(0, int(mutant_num)+2)] += priority
        ans += priority
        # D.add_edge(0, int(mutant_num)+2, priority)
    for mutant_num, priority in wishlist_z.items():
        tmp_dict[(int(mutant_num)+2, 1)] += priority
        ans += priority
        # D.add_edge(int(mutant_num)+2, 1, priority)
    for wish_list in wishlist_mutants:
        if wish_list["squad"] == "Y-Men":
            tmp_dict[(0, int(wish_list["mutant"])+2)] += wish_list["priority"]
            ans += wish_list["priority"]
            # D.add_edge(0, int(wish_list["mutant"])+2, wish_list["priority"])
        else:
            tmp_dict[(int(wish_list["mutant"])+2, 1)] += wish_list["priority"]
            ans += wish_list["priority"]
            # D.add_edge(int(wish_list["mutant"])+2, 1, wish_list["priority"])
    for k, v in tmp_dict.items():
        D.add_edge(k[0], k[1], v)
    for friendship in friendships:
        D.add_edge(int(friendship["friend1"])+2, int(friendship["friend2"])+2, friendship["strength"])
        D.add_edge(int(friendship["friend2"])+2, int(friendship["friend1"])+2, friendship["strength"])
        ans += friendship["strength"]
    max_s = D.flow(0, 1)
    return ans - max_s